# YOLOv26を試す

疑似データ（ランダム画像＋ランダムバウンディングボックス）を生成し、YOLOv26nモデルで5エポック学習して動作確認を行う。

## 目的
- YOLOv26の基本的な使い方を確認する
- 疑似データでのサニティチェック（学習パイプラインが正常に動くか確認）
- 学習済みモデルの保存先を `model_weights/YOLOv26/` に設定する

## 1. セットアップ

In [ ]:
import os
import random
import shutil
from pathlib import Path
from datetime import datetime

import numpy as np
from PIL import Image
from ultralytics import YOLO, settings

# 再現性の確保
random.seed(42)
np.random.seed(42)

## 2. Ultralytics設定の変更

学習済み公開モデルの保存先を `model_weights/YOLOv26/` に変更する。

In [ ]:
# プロジェクトルートを基準にパスを設定
PROJECT_ROOT = Path(os.getcwd()).resolve()

# もしnotebooks/から実行している場合はプロジェクトルートに移動
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
    os.chdir(PROJECT_ROOT)

# モデル重みの保存先
WEIGHTS_DIR = PROJECT_ROOT / "model_weights" / "YOLOv26"
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

# 出力先（実行結果）
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
OUTPUT_DIR = PROJECT_ROOT / "output" / f"{timestamp}_yolov26_pseudo"

# Ultralyticsの設定を更新
# weights_dir: 学習済み公開モデルの自動ダウンロード先
# runs_dir: 学習・推論結果の出力先
settings.update({
    "weights_dir": str(WEIGHTS_DIR),
    "runs_dir": str(OUTPUT_DIR),
})

print(f"プロジェクトルート: {PROJECT_ROOT}")
print(f"重み保存先: {WEIGHTS_DIR}")
print(f"出力先: {OUTPUT_DIR}")
print(f"weights_dir設定: {settings['weights_dir']}")

## 3. 疑似データの生成

YOLO形式のデータセットを疑似的に作成する:  
- ランダムな色のシェイプ（矩形・円）を含む画像を生成
- 対応するYOLO形式のアノテーション（class x_center y_center width height）を作成
- 3クラス: `rectangle`, `circle`, `triangle`

In [ ]:
def generate_pseudo_dataset(
    output_path: Path,
    n_train: int = 50,
    n_val: int = 10,
    img_size: int = 640,
    max_objects: int = 5,
    n_classes: int = 3,
) -> Path:
    """YOLO形式の疑似データセットを生成する。

    Args:
        output_path: データセットの出力先
        n_train: 学習画像の枚数
        n_val: 検証画像の枚数
        img_size: 画像サイズ（正方形）
        max_objects: 1画像あたりの最大オブジェクト数
        n_classes: クラス数

    Returns:
        データセットのルートパス
    """
    class_names = ["rectangle", "circle", "triangle"]

    for split, n_images in [("train", n_train), ("val", n_val)]:
        img_dir = output_path / split / "images"
        lbl_dir = output_path / split / "labels"
        img_dir.mkdir(parents=True, exist_ok=True)
        lbl_dir.mkdir(parents=True, exist_ok=True)

        for i in range(n_images):
            # ランダムな背景色の画像を生成
            bg_color = np.random.randint(0, 128, size=3, dtype=np.uint8)
            img_array = np.full((img_size, img_size, 3), bg_color, dtype=np.uint8)

            # ランダムなノイズを追加
            noise = np.random.randint(0, 30, size=(img_size, img_size, 3), dtype=np.uint8)
            img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)

            n_objects = random.randint(1, max_objects)
            labels = []

            for _ in range(n_objects):
                cls_id = random.randint(0, n_classes - 1)

                # バウンディングボックスをランダムに生成（YOLO形式: x_center, y_center, w, h を0-1で正規化）
                w = random.uniform(0.05, 0.3)
                h = random.uniform(0.05, 0.3)
                x_center = random.uniform(w / 2, 1.0 - w / 2)
                y_center = random.uniform(h / 2, 1.0 - h / 2)

                # 画像にシェイプを描画（視認性を高めるため明るい色で描画）
                x1 = int((x_center - w / 2) * img_size)
                y1 = int((y_center - h / 2) * img_size)
                x2 = int((x_center + w / 2) * img_size)
                y2 = int((y_center + h / 2) * img_size)
                color = np.random.randint(150, 255, size=3).tolist()

                if cls_id == 0:  # 矩形
                    img_array[y1:y2, x1:x2] = color
                elif cls_id == 1:  # 円
                    cy, cx = (y1 + y2) // 2, (x1 + x2) // 2
                    radius = min(x2 - x1, y2 - y1) // 2
                    yy, xx = np.ogrid[:img_size, :img_size]
                    mask = (xx - cx) ** 2 + (yy - cy) ** 2 <= radius ** 2
                    img_array[mask] = color
                else:  # 三角形
                    for row in range(y1, y2):
                        progress = (row - y1) / max(y2 - y1, 1)
                        left = int(x1 + (x2 - x1) * (1 - progress) / 2)
                        right = int(x2 - (x2 - x1) * (1 - progress) / 2)
                        img_array[row, left:right] = color

                labels.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

            # 画像を保存
            img = Image.fromarray(img_array)
            img.save(img_dir / f"{i:04d}.jpg")

            # ラベルを保存
            with open(lbl_dir / f"{i:04d}.txt", "w") as f:
                f.write("\n".join(labels))

    # dataset.yaml を作成
    yaml_content = f"""path: {output_path.resolve()}
train: train/images
val: val/images

names:
  0: rectangle
  1: circle
  2: triangle
"""
    yaml_path = output_path / "dataset.yaml"
    yaml_path.write_text(yaml_content, encoding="utf-8")

    print(f"データセット生成完了:")
    print(f"  学習画像: {n_train}枚")
    print(f"  検証画像: {n_val}枚")
    print(f"  クラス: {class_names}")
    print(f"  保存先: {output_path}")

    return yaml_path

In [ ]:
# 疑似データセットを生成
DATASET_DIR = PROJECT_ROOT / "data" / "pseudo_yolo_dataset"

# 既存のデータがあれば削除して再生成
if DATASET_DIR.exists():
    shutil.rmtree(DATASET_DIR)

dataset_yaml = generate_pseudo_dataset(DATASET_DIR, n_train=50, n_val=10)

## 4. 生成データの確認

生成した疑似データの一部を可視化して、正しく作成されているか確認する。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_samples(dataset_dir: Path, n_samples: int = 4) -> None:
    """データセットのサンプルを可視化する。

    Args:
        dataset_dir: データセットのルートパス
        n_samples: 表示するサンプル数
    """
    img_dir = dataset_dir / "train" / "images"
    lbl_dir = dataset_dir / "train" / "labels"
    class_names = ["rectangle", "circle", "triangle"]
    colors = ["red", "blue", "green"]

    img_files = sorted(img_dir.glob("*.jpg"))[:n_samples]
    fig, axes = plt.subplots(1, n_samples, figsize=(4 * n_samples, 4))
    if n_samples == 1:
        axes = [axes]

    for ax, img_file in zip(axes, img_files):
        img = Image.open(img_file)
        ax.imshow(img)

        lbl_file = lbl_dir / img_file.with_suffix(".txt").name
        if lbl_file.exists():
            with open(lbl_file) as f:
                for line in f:
                    parts = line.strip().split()
                    cls_id = int(parts[0])
                    x_c, y_c, w, h = map(float, parts[1:])
                    # YOLO形式からピクセル座標に変換
                    img_w, img_h = img.size
                    x1 = (x_c - w / 2) * img_w
                    y1 = (y_c - h / 2) * img_h
                    rect = patches.Rectangle(
                        (x1, y1), w * img_w, h * img_h,
                        linewidth=2, edgecolor=colors[cls_id], facecolor="none"
                    )
                    ax.add_patch(rect)
                    ax.text(x1, y1 - 5, class_names[cls_id],
                            color=colors[cls_id], fontsize=8, weight="bold")

        ax.set_title(img_file.name)
        ax.axis("off")

    plt.tight_layout()
    plt.show()

visualize_samples(DATASET_DIR)

## 5. YOLOv26モデルの読み込みと学習

学習済みYOLOv26nモデルを読み込み、疑似データで5エポック学習する。  
モデルの重みは `model_weights/YOLOv26/` に自動ダウンロード・保存される。

In [ ]:
# 学習済みYOLOv26nモデルを読み込む
# weights_dir設定により、"yolo26n.pt" は model_weights/YOLOv26/ から参照される
# 初回実行時は自動的に model_weights/YOLOv26/yolo26n.pt にダウンロードされる
model = YOLO("yolo26n.pt")

# モデル情報を表示
model.info()

# 読み込みパスを確認
print(f"\nモデル読み込みパス: {model.ckpt_path}")

In [ ]:
# 重みファイルの保存場所を確認
print("重みファイルの保存先:")
for f in WEIGHTS_DIR.glob("*.pt"):
    print(f"  {f.name} ({f.stat().st_size / 1024 / 1024:.1f} MB)")

In [ ]:
# 疑似データで5エポック学習
results = model.train(
    data=str(dataset_yaml),
    epochs=5,
    imgsz=640,
    batch=8,
    device="cpu",  # GPUがあれば "0" に変更
    workers=0,     # Windowsでのマルチプロセス問題を回避
    verbose=True,
    project=str(OUTPUT_DIR),
    name="train",
    exist_ok=True,
)

## 6. 学習結果の確認

In [ ]:
# 学習結果の確認
print("=== 学習結果 ===")
print(f"出力先: {OUTPUT_DIR}")

# 出力ファイル一覧
train_dir = OUTPUT_DIR / "train"
if train_dir.exists():
    print("\n生成されたファイル:")
    for f in sorted(train_dir.rglob("*")):
        if f.is_file():
            size_kb = f.stat().st_size / 1024
            print(f"  {f.relative_to(train_dir)} ({size_kb:.1f} KB)")

In [ ]:
# 学習曲線の可視化（結果画像がある場合）
results_img = train_dir / "results.png"
if results_img.exists():
    img = Image.open(results_img)
    plt.figure(figsize=(16, 8))
    plt.imshow(img)
    plt.axis("off")
    plt.title("YOLOv26n - 学習結果")
    plt.show()
else:
    print("結果画像が見つかりません")

## 7. 推論テスト

学習したモデルで検証画像に対して推論を実行し、結果を確認する。

In [ ]:
# 学習済みモデルで推論テスト
best_model_path = train_dir / "weights" / "best.pt"
if best_model_path.exists():
    trained_model = YOLO(str(best_model_path))
else:
    # best.ptがなければlast.ptを使用
    last_model_path = train_dir / "weights" / "last.pt"
    trained_model = YOLO(str(last_model_path))

# 検証画像で推論
val_images = sorted((DATASET_DIR / "val" / "images").glob("*.jpg"))[:4]
results = trained_model.predict(
    source=[str(img) for img in val_images],
    conf=0.25,
    save=False,
    verbose=False,
)

# 推論結果を表示
class_names = ["rectangle", "circle", "triangle"]
fig, axes = plt.subplots(1, len(val_images), figsize=(4 * len(val_images), 4))
if len(val_images) == 1:
    axes = [axes]

for ax, result, img_path in zip(axes, results, val_images):
    # 結果を画像にプロット
    annotated = result.plot()
    # BGRからRGBに変換
    annotated = annotated[:, :, ::-1]
    ax.imshow(annotated)
    n_detections = len(result.boxes)
    ax.set_title(f"{img_path.name} ({n_detections}検出)")
    ax.axis("off")

plt.tight_layout()
plt.show()

print(f"\n推論完了: {len(val_images)}枚の画像を処理")

## まとめ

- YOLOv26nモデルの学習済み重みを `model_weights/YOLOv26/` にダウンロード・保存した
- 疑似データ（3クラス: rectangle, circle, triangle）で5エポック学習を実行した
- 学習結果は `output/YYYYMMDD-HHMMSS_yolov26_pseudo/` に出力された

### 注意事項
- 疑似データのため精度は低いが、学習パイプラインの動作確認が目的
- GPUが利用可能な場合は `device="0"` に変更すると高速化できる
- 実データでの学習時は画像枚数・エポック数・バッチサイズを適切に調整すること